In [ ]:
#lendo o conjunto de dados

import pandas as pd


data = pd.read_csv("../input/leadscoring/Leads.csv")

data

y=data.Converted
X = data.drop(['Lead Number','Converted','Prospect ID'], axis=1)

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
categorical_cols

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
#fitando
#from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#model = RandomForestClassifier(n_estimators=100, random_state=0)
model = XGBClassifier()
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train,y_train)

In [ ]:
preds = my_pipeline.predict(X_valid)
preds

In [ ]:
#valorando o modelo



from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

score = mean_absolute_error(y_valid, preds)
acscore = accuracy_score(y_valid,preds)
matrix = confusion_matrix(y_valid,preds)
f1 = f1_score(y_valid,preds)

print('MAE:',score)
print('Accuracy:',acscore)
print('f1: ',f1)
print('Matrix:',matrix)

print('Precision is: ', precision_score(y_valid, preds))
print('Recall is: ', recall_score(y_valid, preds))

In [ ]:
#Plotando a matriz de confusão
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(matrix) 

disp.plot()